In [2]:
import pandas as pd
user_target = pd.read_csv("target_users.tsv")
item = pd.read_csv("item_history.tsv",sep="\t")


In [3]:
item.shape

(6319946, 4)

In [4]:
item[item.user_id.isin(user_target.user_id)].shape

(2456196, 4)

In [5]:
item[~item.user_id.isin(user_target.user_id)].user_id.nunique()

690442

In [6]:
item_ids = item.item_id.unique()
user_ids = item.user_id.unique()
item2idx = {item:idx for idx,item in enumerate(item_ids)}
idx2item = {idx:item for item,idx in item2idx.items()}
user2idx = {user:idx for idx,user in enumerate(user_ids)}
idx2user = {idx:user for idx,user in enumerate(user_ids)}

In [7]:
item.item_id.max(),len(item.item_id.unique()),len(user2idx),len(item2idx)

('I99999', 328823, 913707, 328823)

In [8]:
max([int(x[1:]) for x in item.item_id.values]), len(item2idx)

(355712, 328823)

In [ ]:
# too much memory
# import numpy as np
# np.zeros([len(user2idx),len(item2idx)])

### user feature

In [9]:
user_master = pd.read_csv("user_master.tsv",sep="\t")

In [10]:
user_master.shape,user_master.user_id.max(),user_master.user_id.nunique()

((1640956, 16), 1640956, 1640956)

In [11]:
user_feature = user_master[user_master.user_id.isin(set(user_ids).union(set(user_target.user_id)))]
user_feature = user_feature.reset_index(drop=True)
user_feature.shape

(940785, 16)

In [12]:
# user_feature = user_feature[["user_id"]]
user_feature.shape

(940785, 16)

In [13]:
user_feature = pd.concat([user_feature,pd.get_dummies(user_feature["feature1"])],axis=1)
user_feature = pd.concat([user_feature,pd.get_dummies(user_feature["feature2"])],axis=1)
user_feature = pd.concat([user_feature,pd.get_dummies(user_feature["feature3"])],axis=1)
user_feature = pd.concat([user_feature,pd.get_dummies(user_feature["feature4"])],axis=1)
user_feature = pd.concat([user_feature,pd.get_dummies(user_feature["feature5"])],axis=1)
user_feature = pd.concat([user_feature,pd.get_dummies(user_feature["feature6"])],axis=1)

In [14]:
user_feature.columns

Index(['user_id', 'feature1', 'feature2', 'feature3', 'feature4', 'feature5',
       'feature6', 'feature_list1', 'feature_list2',
       'feature_list2_frequencies',
       ...
       'F1', 'F10', 'F2', 'F3', 'F4', 'F5', 'F6', 'F7', 'F8', 'F9'],
      dtype='object', length=102)

In [15]:
user_feature["A1"].sum(),user_feature["A2"].sum(),user_feature["A3"].sum()

(691693, 249091, 1)

In [16]:
from sklearn.preprocessing import MultiLabelBinarizer
import numpy as np
def multihot_encode(user_fea, target, topk = 9999):
    feature_list = user_fea[target].apply(lambda x:str(x).split(',')).values
    feature_list_multihot = MultiLabelBinarizer().fit_transform(feature_list)
    common_feature_idx = np.argsort(feature_list_multihot.sum(axis=0))[-topk:]
    common_feature_idx = common_feature_idx[::-1]
    # print(feature_list_multihot.sum(axis=0)[common_feature_idx])
    # feature_list_multihot = feature_list_multihot.astype(np.bool_)
    extra_feature = pd.DataFrame(feature_list_multihot[:,common_feature_idx],dtype=bool)
    extra_feature.columns=[target+"_"+str(ii) for ii in range(extra_feature.shape[1])]
    # print(extra_feature.memory_usage())
    return extra_feature

KeyboardInterrupt: 

In [ ]:
MultiLabelBinarizer().fit_transform(user_feature["feature_list2"].apply(lambda x:str(x).split(',')).values).shape

In [ ]:
parsed_list3 = user_feature["feature_list3"].apply(lambda x:str(x).split(',')).values
parsed_list2 = user_feature["feature_list2"].apply(lambda x:str(x).split(',')).values
parsed_list4 = user_feature["feature_list4"].apply(lambda x:str(x).split(',')).values
parsed_list5 = user_feature["feature_list5"].apply(lambda x:str(x).split(',')).values

In [ ]:
list_fea = [y for x in parsed_list2 for y in x]
len(set(list_fea))

66837

In [ ]:
for i in [1,3,4]:
    feature_list_multihot = multihot_encode(user_feature,f"feature_list{i}",10)
    user_feature = pd.concat([user_feature,feature_list_multihot],axis=1)

In [ ]:
# user_feature

In [ ]:
list(user_feature.columns)

['user_id',
 'feature1',
 'feature2',
 'feature3',
 'feature4',
 'feature5',
 'feature6',
 'feature_list1',
 'feature_list2',
 'feature_list2_frequencies',
 'feature_list3',
 'feature_list3_frequencies',
 'feature_list4',
 'feature_list4_frequencies',
 'feature_list5',
 'feature_list5_frequencies',
 'A1',
 'A2',
 'A3',
 'B1',
 'B10',
 'B11',
 'B12',
 'B13',
 'B14',
 'B15',
 'B16',
 'B17',
 'B18',
 'B19',
 'B2',
 'B20',
 'B21',
 'B22',
 'B23',
 'B24',
 'B25',
 'B26',
 'B27',
 'B28',
 'B29',
 'B3',
 'B30',
 'B31',
 'B32',
 'B33',
 'B34',
 'B35',
 'B36',
 'B37',
 'B38',
 'B39',
 'B4',
 'B40',
 'B41',
 'B42',
 'B43',
 'B44',
 'B45',
 'B46',
 'B47',
 'B48',
 'B5',
 'B6',
 'B7',
 'B8',
 'B9',
 'C1',
 'C10',
 'C11',
 'C2',
 'C3',
 'C4',
 'C5',
 'C6',
 'C7',
 'C8',
 'C9',
 'D1',
 'D2',
 'E1',
 'E10',
 'E11',
 'E12',
 'E2',
 'E3',
 'E4',
 'E5',
 'E6',
 'E7',
 'E8',
 'E9',
 'F1',
 'F10',
 'F2',
 'F3',
 'F4',
 'F5',
 'F6',
 'F7',
 'F8',
 'F9',
 'feature_list1_0',
 'feature_list1_1',
 'feature_lis

In [ ]:
import importlib
import BPR_lib
importlib.reload(BPR_lib)
from BPR_lib import reduce_mem
user_feature = reduce_mem(user_feature)

### hist / last

In [18]:
item = item.sort_values(by=['user_id', 'latest_timestamp'])
item_last = item.groupby("user_id").tail(1)
def hist_func(user_df):
    if len(user_df) == 1:
        return user_df
    else:
        return user_df[:-1]
from tqdm import tqdm
tqdm.pandas()

# item_hist = item.groupby('user_id').progress_apply(hist_func).reset_index(drop=True)

In [19]:
# item_hist.to_csv("item_hist.csv",index=False)
item_hist = pd.read_csv("item_hist.csv",index_col=None)

In [12]:
item_hist.shape, item_last.shape, item.shape

((5525076, 4), (913707, 4), (6319946, 4))

In [13]:
check = item_last.merge(item_hist,on=["user_id", "item_id"])
check.shape, check[check.latest_timestamp_x == check.latest_timestamp_y].shape

((118837, 6), (118837, 6))

In [14]:
item[item.frequency!=1].sort_values(["user_id", "item_id"])

,user_id,item_id,latest_timestamp,frequency
6005183,51,I145321,1584921600,2
6005184,51,I333846,1584921600,2
6005180,51,I72579,1584921600,2
6005181,51,I78233,1584921600,2
2273050,95,I202068,1582934400,2
...,...,...,...,...
1448553,1640857,I77077,1583452800,2
2811960,1640889,I232856,1587945600,2
2811967,1640889,I308948,1588032000,2
2241527,1640913,I104881,1585180800,2


In [15]:
item.set_index(["user_id","item_id"]).loc[51].loc["I145321"]

latest_timestamp    1584921600
frequency                    2
Name: I145321, dtype: int64

In [16]:
item_last.latest_timestamp.mean(), item_hist.latest_timestamp.mean()

(1585465311.0250878, 1584377814.3402915)

In [17]:
item_hist

,user_id,item_id,latest_timestamp,frequency
0,2,I89717,1581897600,1
1,3,I311781,1585872000,1
2,3,I286302,1585872000,1
3,3,I304391,1585872000,1
4,3,I354326,1585872000,1
...,...,...,...,...
5525071,1640954,I225457,1583539200,1
5525072,1640954,I225458,1583539200,1
5525073,1640954,I222174,1583539200,1
5525074,1640955,I134557,1585440000,1


In [18]:
item_hist.item_id.nunique(),item.item_id.nunique()

(312038, 328823)

In [19]:
item.latest_timestamp.max() - item.latest_timestamp.min()
item.latest_timestamp.max() ,item.latest_timestamp.min()

(1588118400, 1580428800)

In [20]:
user_feature.shape, item.shape,user_target.shape

NameError: name 'user_feature' is not defined

In [ ]:
popular_items = item_hist.groupby("item_id")["latest_timestamp"].count()\
    .sort_values(ascending=False)
popular_2 = item_hist["item_id"].value_counts()[:]
popular_items[:10],popular_2[:10]

(item_id
 I102695    20798
 I103751    19440
 I302808    19390
 I177269    18896
 I31876     15219
 I46072     14590
 I337068    14415
 I193280    12601
 I196644    11955
 I7453      11882
 Name: latest_timestamp, dtype: int64,
 item_id
 I102695    20798
 I103751    19440
 I302808    19390
 I177269    18896
 I31876     15219
 I46072     14590
 I337068    14415
 I193280    12601
 I196644    11955
 I7453      11882
 Name: count, dtype: int64)

In [ ]:
item_hist.user_id.max()

NameError: name 'item_hist' is not defined

### train valid split

In [20]:
from sklearn.model_selection import GroupKFold
kFold = GroupKFold(n_splits=5)

for i, (trn_idx, val_idx) in enumerate(kFold.split(item_hist,item_hist,groups=item_hist["user_id"])):
    item_trn = item_hist.iloc[trn_idx]
    item_val = item_hist.iloc[val_idx]
    break


In [358]:
item_trn.user_id.nunique(),item_val.user_id.nunique(),\
  item_trn.item_id.nunique(),item_val.user_id.nunique()

(730966, 182741, 299158, 182741)

### make dataset

In [52]:

N = 150000
# top_items = list(item["item_id"].value_counts().index[:N].values)
top_items = [item2idx[x] for x in item.item_id.values]
# N_U = 450000
# top_users = list(item["user_id"].value_counts().index[:N_U].values)

train_data_base = item_trn[["user_id","item_id"]]
train_data_base.loc[:,"item_id"] = train_data_base.item_id.apply(lambda x:item2idx[x])

# train_data_base = train_data_base[train_data_base.item_id.isin(top_items)]
# train_data_base = train_data_base[train_data_base.user_id.isin(top_users)]

# print("choosing top", N,N_U, item.shape[0],train_data_base.shape[0])

In [149]:
import random
import numpy as np

train_list = []
for neg_num in range(10):
    print(neg_num, "...")
    train_data_base.loc[:,"neg_item_id"] = np.random.choice(top_items,train_data_base.shape[0])
    # train_data_base.loc[:,"neg_item_id"] = train_data_base.neg_item_id.apply(lambda x:item2idx[x])
    train_list.append(train_data_base.copy())
train_data = pd.concat(train_list)
train_data.head(5)

0 ...
1 ...
2 ...
3 ...
4 ...
5 ...
6 ...
7 ...
8 ...
9 ...


,user_id,item_id,neg_item_id
1,3,17221,5179
2,3,3770,764
3,3,17225,233
4,3,16696,190877
5,5,22165,73762


In [160]:
train_data.item_id.value_counts().describe()

count    283142.000000
mean        156.107536
std        1473.077678
min          10.000000
25%          10.000000
50%          20.000000
75%          50.000000
max      166090.000000
Name: count, dtype: float64

In [162]:
train_data_base.item_id.value_counts().describe()

count    283142.000000
mean         15.610754
std         147.307768
min           1.000000
25%           1.000000
50%           2.000000
75%           5.000000
max       16609.000000
Name: count, dtype: float64

In [44]:
train_data.shape, train_data_base.shape

((44200600, 3), (4420060, 3))

In [69]:
print(train_data.shape, train_data.user_id.nunique(),train_data.item_id.nunique())
print(train_data.neg_item_id.nunique(), train_data_base.shape)
print(train_data[["user_id","item_id"]].duplicated().sum(),\
            train_data[["user_id","item_id","neg_item_id"]].duplicated().sum())

(44200600, 3) 730966 283142
328823 (4420060, 3)


KeyboardInterrupt: 

In [51]:
train_data_base.item_id.value_counts().describe()

count    283142.000000
mean         15.610754
std         147.307768
min           1.000000
25%           1.000000
50%           2.000000
75%           5.000000
max       16609.000000
Name: count, dtype: float64

In [ ]:
train_data_base.user_id.value_counts().describe()

count    199754.000000
mean         13.729923
std          13.007129
min           1.000000
25%           9.000000
50%          11.000000
75%          15.000000
max        1055.000000
Name: count, dtype: float64

In [349]:
item.item_id.value_counts().describe()

count    328823.000000
mean         19.219903
std         196.719348
min           1.000000
25%           1.000000
50%           2.000000
75%           6.000000
max       23663.000000
Name: count, dtype: float64

In [ ]:
train_data.head()

,user_id,item_id,neg_item_id
0,484,0,108872
1,484,1,13764
2,815,2,108122
3,998,3,121519
4,998,4,261507


In [ ]:
train_data.item_id.value_counts().sort_values(ascending=False)[149999:]

item_id
44052    8
Name: count, dtype: int64

In [ ]:
train_data.user_id.nunique(),item_val.user_id.nunique()

(908075, 182741)

In [58]:
train_data.merge(item_val,on="user_id")


,user_id,item_id_x,neg_item_id,item_id_y,latest_timestamp,frequency


In [ ]:
train_data.iloc[:10].sample(5)

,user_id,item_id,neg_item_id
5,998,5,147028
6,998,6,7095
4,998,4,171863
0,484,0,44927
2,815,2,8695


In [ ]:

train_data_sampled = train_data[train_data.columns[:]].sample(frac=0.3)#5000000)
train_data_sampled.shape, train_data_sampled.head()

((4113910, 3),
          user_id item_id neg_item_id
 2950544   513715   38231       16856
 2821233   462285    6143       43778
 5811418  1378496    3142       88469
 6194097      913   29135       31610
 3679050   578853    1359       20243)

In [ ]:
train_data_sampled.user_id.value_counts().describe()

count    199598.000000
mean         20.610978
std          19.865232
min           1.000000
25%          12.000000
50%          16.000000
75%          23.000000
max        1654.000000
Name: count, dtype: float64

In [ ]:
train_data_sampled.item_id.value_counts().describe()

count    30000.000000
mean       137.130333
std        422.829663
min          1.000000
25%         23.000000
50%         38.000000
75%         86.000000
max      12882.000000
Name: count, dtype: float64

In [ ]:
user_target.user_id.max()

1640953

In [317]:
user_feature = user_feature.set_index("user_id")

# fea_cols = list(user_feature.columns[1:7]) + list(user_feature.columns[16:])
fea_cols = list(user_feature.columns[15:])
user_feature[fea_cols].head()

NameError: name 'user_feature' is not defined

In [ ]:
fea_cols

['A1',
 'A2',
 'A3',
 'B1',
 'B10',
 'B11',
 'B12',
 'B13',
 'B14',
 'B15',
 'B16',
 'B17',
 'B18',
 'B19',
 'B2',
 'B20',
 'B21',
 'B22',
 'B23',
 'B24',
 'B25',
 'B26',
 'B27',
 'B28',
 'B29',
 'B3',
 'B30',
 'B31',
 'B32',
 'B33',
 'B34',
 'B35',
 'B36',
 'B37',
 'B38',
 'B39',
 'B4',
 'B40',
 'B41',
 'B42',
 'B43',
 'B44',
 'B45',
 'B46',
 'B47',
 'B48',
 'B5',
 'B6',
 'B7',
 'B8',
 'B9',
 'C1',
 'C10',
 'C11',
 'C2',
 'C3',
 'C4',
 'C5',
 'C6',
 'C7',
 'C8',
 'C9',
 'D1',
 'D2',
 'E1',
 'E10',
 'E11',
 'E12',
 'E2',
 'E3',
 'E4',
 'E5',
 'E6',
 'E7',
 'E8',
 'E9',
 'F1',
 'F10',
 'F2',
 'F3',
 'F4',
 'F5',
 'F6',
 'F7',
 'F8',
 'F9',
 'feature_list1_0',
 'feature_list1_1',
 'feature_list1_2',
 'feature_list1_3',
 'feature_list1_4',
 'feature_list1_5',
 'feature_list1_6',
 'feature_list1_7',
 'feature_list1_8',
 'feature_list1_9',
 'feature_list1_0',
 'feature_list1_1',
 'feature_list1_2',
 'feature_list1_3',
 'feature_list1_4',
 'feature_list1_5',
 'feature_list1_6',
 'feature_lis

In [ ]:
num_fea = len(fea_cols)
train_data = train_data.merge(user_feature[fea_cols],on="user_id")


In [150]:
import importlib
import BPR_lib
importlib.reload(BPR_lib)
from BPR_lib import BPRData
import torch.utils.data as data
train_dataset = BPRData(train_data.values, True)
train_loader = data.DataLoader(train_dataset,batch_size=1024,shuffle=True,num_workers=0)


In [32]:
train_data.user_id.nunique()

730966

In [41]:
len(train_dataset),train_dataset[45687],

(44200600, (16878, 281, 104515))

In [ ]:
for user_fea,item_i,item_j in tqdm(train_loader):
    item_i.cuda()
    # print(user_fea.shape,item_i)
    # print(item_i.dtype)

In [ ]:
train_data.iloc[0].values.astype(np.float32)[3:].shape

(0,)

In [ ]:
user_feature.shape[1],train_dataset[0][0].shape

(112, ())

In [ ]:
np.sum(train_data.iloc[0].values[3:].astype(np.float32)!=train_dataset[0][0])

0

In [372]:
train_data.neg_item_id.nunique(),train_data.item_id.nunique(),\
    train_data.user_id.nunique(), train_data.user_id.max(),

(150000, 147368, 359925, 1640954)

### Training

In [21]:
import importlib
import BPR_lib
importlib.reload(BPR_lib)
from BPR_lib import BPR,BPR2
import torch.optim as optim
import torch
factor_num = 64
# model1 = BPR(user_num, click_num, factor_num)

model = BPR(item.user_id.max()+1, len(item_ids), factor_num)
# model = BPR2(num_fea,len(item_ids),factor_num)
args_lambda = 0.001
args_lr = 0.01
model.cuda()
optimizer = optim.SGD(model.parameters(), lr=args_lr, weight_decay=args_lambda)

In [ ]:
train_data.user_id.max(),item.user_id.max(),user_target.user_id.max()

(1640954, 1640955, 1640953)

In [30]:

pre_i,pre_j = model(torch.tensor(train_dataset[0][0]).cuda(),\
      torch.tensor(train_dataset[0][1]).cuda(),torch.tensor(train_dataset[0][2]).cuda())
pre_i,pre_j

(tensor(-0.0003, device='cuda:0', grad_fn=<SumBackward1>),
 tensor(-0.0003, device='cuda:0', grad_fn=<SumBackward1>))

In [ ]:
len(train_data),len(train_loader)

(27426070, 26784)

In [ ]:
torch.tensor([-2,-3,-4]).sigmoid().log().sum(),torch.tensor([2,3,4]).sigmoid().log().sum()

tensor(-9.1937)

In [151]:
from tqdm import tqdm

ii = 0
for epoch in range(1):
    model.train()
    model.cuda()
    for user_fea,item_i,item_j in tqdm(train_loader):
        user_fea = user_fea.cuda()
        item_i = item_i.cuda()
        item_j = item_j.cuda()
        # print(user_fea.shape,item_i)
        model.zero_grad()
        pred_i,pred_j = model(user_fea,item_i,item_j)
        ii+=1
        if ii % 1000 == 0:
            print("check", [list[x.detach().cpu().numpy()] for x in [pred_i[:5],pred_j[:5],\
            model.user_embed(user_fea)[0][:10],model.item_embed(item_i)[0][:10]]])
        loss = - (pred_i - pred_j).sigmoid().log().sum()
        loss.backward()
        optimizer.step()
    # print(u_embed.shape,embed_i.shape)

100%|█████████▉| 43010/43165 [12:26<00:02, 61.62it/s]

check [list[array([ 0.00522128,  1.5333514 ,  2.1593952 ,  0.585479  , -0.00701998],
      dtype=float32)], list[array([-1.9821422 ,  0.11736298, -2.3245752 ,  0.28544337, -0.00242912],
      dtype=float32)], list[array([ 0.00503462,  0.00802071,  0.00068001,  0.02247093,  0.02310743,
        0.03605204, -0.04406163,  0.03588691, -0.01415795,  0.02315615],
      dtype=float32)], list[array([-0.01132192,  0.01105928,  0.0201349 ,  0.00762337,  0.00472754,
       -0.00657688,  0.01413851,  0.00225606,  0.00123782,  0.01599493],
      dtype=float32)]]


100%|██████████| 43165/43165 [12:34<00:00, 57.23it/s]


In [52]:
trn_user_ids = set()
for user_fea,item_i,item_j in tqdm(train_loader):
    trn_user_ids.update(user_fea.numpy())

  8%|▊         | 3244/43165 [00:20<04:12, 158.14it/s]


KeyboardInterrupt: 

In [51]:
trn_user_ids.update(user_fea.numpy())

In [47]:
model.user_embed(torch.tensor(item_val.iloc[41000].user_id).cuda()).detach().cpu().numpy()

array([ 6.8908827e-03,  5.5735814e-03, -9.2331143e-03,  4.6274462e-03,
        4.4738078e-03,  5.1760599e-03, -3.7225938e-04,  8.3511537e-03,
        1.2542726e-02,  2.5386419e-03, -8.4746145e-03,  3.9174948e-03,
       -2.1102675e-03,  6.2068054e-03, -1.3101082e-03,  4.3848511e-03,
       -3.5490554e-03,  9.0378366e-04, -7.8911530e-03, -1.1393058e-03,
        7.4744308e-03, -3.5606534e-03, -1.9727172e-03,  3.2889291e-03,
        5.1537985e-03,  2.3442300e-03, -1.2512995e-03,  6.6579049e-03,
        5.6610839e-03,  9.4328672e-05,  5.8099339e-03,  5.1901774e-03,
        3.1130626e-03, -4.7986233e-03, -6.2107923e-03, -4.0074475e-03,
        8.2914541e-03,  4.8420997e-03,  4.6810796e-03, -5.1157684e-03,
       -3.4454253e-03, -5.7737539e-03,  5.4337289e-03,  6.4413501e-03,
       -1.5137944e-02, -1.3992387e-02, -8.7928735e-03,  2.1315119e-04,
       -2.6681249e-03,  9.1972584e-03, -3.0238536e-04, -1.0430278e-03,
        1.5965941e-03, -5.2361116e-03,  3.8958158e-04,  6.4978208e-03,
      

In [104]:
model.user_embed(torch.tensor(item_trn.iloc[41000].user_id).cuda()).detach().cpu().numpy()

array([-6.47718844e-04,  2.44897697e-03, -5.82386553e-03, -7.07108702e-04,
       -1.72569312e-03, -3.00709298e-03, -3.77733330e-03, -2.36914004e-03,
       -2.03323201e-03,  2.39809300e-03,  4.52667614e-03, -1.26563162e-02,
        4.01852652e-04,  8.27247370e-03,  8.27635173e-03, -1.41783594e-03,
       -8.02599080e-03, -8.52880441e-03,  6.55439030e-03, -3.73293622e-03,
       -2.76638952e-04, -8.03745538e-03,  1.08552836e-02,  1.32788962e-03,
       -2.76707485e-03,  5.13987150e-03, -6.37872610e-03, -1.01398053e-02,
        6.66447729e-03,  1.36400415e-02, -4.31851577e-03,  5.89336315e-03,
        2.93145044e-04, -3.33363330e-03,  2.17886921e-03, -5.45824878e-03,
        4.84044943e-03, -5.93662076e-03, -1.07976440e-02,  4.05566860e-03,
       -1.96400601e-02, -8.24945862e-04,  6.45932555e-03, -4.14338137e-05,
        4.50625457e-03,  8.22412374e-04,  4.87117330e-03,  4.32155188e-03,
       -6.81279507e-03, -1.08568136e-04, -9.60117765e-03, -1.30240880e-02,
       -8.50398187e-03, -

In [369]:
model.user_embed(torch.tensor(9998).cuda()).detach().cpu().numpy()

array([-4.2816708e-03,  6.0236640e-03, -5.9078990e-05,  2.7680367e-03,
       -4.6667904e-03, -2.5506539e-03, -4.1887364e-03, -4.8868647e-03,
        6.2343157e-03,  3.7692725e-03, -2.8563896e-03, -3.5775490e-03,
        6.0844012e-03,  1.1207264e-02,  3.2449367e-03,  5.2068280e-03,
       -3.0789032e-04,  1.3829237e-03,  2.0116535e-03,  4.2341845e-03,
        7.2334404e-03, -6.4414861e-03, -9.0164654e-03,  3.1441250e-03,
        3.9684894e-03,  9.3372883e-03, -2.3234335e-03, -1.6447932e-03,
        3.6622146e-03, -1.2775267e-02,  2.5935273e-03, -4.6908297e-03,
        9.0169767e-03, -5.5137817e-03, -1.9935211e-03,  4.1265134e-03,
        6.1708940e-03, -4.5954944e-03, -1.5023616e-03, -4.2850189e-03,
        2.4073434e-03,  3.2268072e-04,  7.5593627e-05, -1.0880918e-02,
       -5.5108508e-03,  5.9238039e-03,  1.6519962e-03, -1.1228019e-02,
       -5.4894718e-03, -1.8805348e-03, -3.0495867e-04,  2.4685783e-03,
       -2.5832229e-03,  6.1180112e-03, -3.3032186e-03,  1.2613136e-03,
      

In [333]:
train_data

,user_id,item_id,neg_item_id
3,998,3,67941
4,998,4,16414
5,998,5,11396
6,998,6,24734
7,998,7,45898
...,...,...,...
6319941,1640781,251,73420
6319942,1640781,85,58763
6319943,1640781,120,180127
6319944,1640781,356,61921


In [22]:
# torch.save(model.state_dict(),"BPR_20w_3w_sample10_factor64.pth")
model.load_state_dict(torch.load("BPR_sample10_factor64_ep3.pth"))

<All keys matched successfully>

In [ ]:
user_fea.shape

torch.Size([1024])

In [152]:
torch.save(model.state_dict(),"BPR_sample10_factor64_ep3.pth")

In [63]:
idx2item[int(item_i[0].numpy())],item_i[0]

('I4701', tensor(1480))

In [23]:
start = 1000000

for ii in range(start, start+20):
    uu,item_i,item_j = train_dataset[ii]
    uu = torch.tensor(uu).cuda()
    item_fea_i = torch.tensor(item_i).cuda()
    item_fea_j = torch.tensor(item_j).cuda()
    pred_i2,pred_j2 = model(uu,item_fea_i,item_fea_j)
    print([float(x.detach().cpu().numpy()) for x in [pred_i2,pred_j2]])

NameError: name 'train_dataset' is not defined

In [110]:
start = 21000000
model.cuda()
for ii in range(start, start+20):
    uu,item_i,item_j = train_dataset[ii]
    uu = torch.tensor(uu).cuda()
    item_fea_i = torch.tensor(item_i).cuda()
    item_fea_j = torch.tensor(item_j).cuda()
    pred_i2,pred_j2 = model(uu,item_fea_i,item_fea_j)
    print([float(x.detach().cpu().numpy()) for x in [pred_i2,pred_j2]])

[0.6474443674087524, 0.07947587966918945]
[1.7974094152450562, 0.0660865306854248]
[0.08540710061788559, -0.8419563174247742]
[0.8017714023590088, 0.17572204768657684]
[-0.08387137204408646, 0.016119753941893578]
[0.8485485315322876, -1.7725920677185059]
[0.07586459070444107, -0.012445572763681412]
[0.25201013684272766, 0.004813140258193016]
[0.00676204776391387, 0.01935204118490219]
[0.8383329510688782, -0.022735022008419037]
[0.051315151154994965, -0.9918891191482544]
[4.365927696228027, -0.2936568260192871]
[4.364213943481445, -0.4521724283695221]
[4.079098224639893, -0.08155763149261475]
[3.6191165447235107, 0.46548599004745483]
[3.328695774078369, -0.07319173216819763]
[0.006003241054713726, -0.005468173883855343]
[-0.043182529509067535, -0.11877325922250748]
[0.46940988302230835, 0.012620078399777412]
[0.11884622275829315, 0.09699103236198425]


In [25]:
from tqdm import tqdm
all_items= [item2idx[x] for x in item_ids]
trn_item_embeds = []

for ii in tqdm(range(0,len(all_items),64)):
    # item_id_idx = [item2idx[x] for x in top_items[ii:ii+64]]
    item_embed = model.item_embed(torch.tensor(all_items[ii:ii+64],dtype=torch.int32).cuda())
    trn_item_embeds.extend(item_embed.detach().cpu().numpy())
len(trn_item_embeds)
import numpy as np
import faiss
index = faiss.IndexFlatIP(factor_num)
index.add(np.array(trn_item_embeds))


100%|██████████| 5138/5138 [00:02<00:00, 2029.98it/s]


In [290]:
len(trn_item_embeds), trn_item_embeds[0].shape

(150000, (64,))

In [39]:
item_val.item_id.nunique(),item_val.user_id.nunique(),item_val.shape

(146418, 182741, (1105016, 4))

In [26]:
test_user_ids = list(item_val.user_id.unique())
gt_item = item_val.groupby("user_id")["item_id"].apply(list)
gt_item2 = item_last[item_last.user_id.isin(test_user_ids)].set_index("user_id")
np.random.shuffle(test_user_ids)
count1 =0 
count2 = 0
total1 = 0
total2 = 0
check = 0
with tqdm(total=len(test_user_ids)) as pbar:
    for test_user in test_user_ids:
        user_embed = model.user_embed(torch.tensor(test_user).cuda()).detach().cpu().numpy()
        # sims2 = np.dot(user_embed,np.array(trn_item_embeds).T)
        # arts_fetch2 = np.random.choice(trn_item_ids,30000)
        # arts_fetch1 = np.array(top_items)[np.argsort(sims2)[-1500:]]
        # arts_fetch2 = np.array(top_items)[np.argsort(sims2)[:148500]]
        sim,idx =  index.search(np.array([user_embed]),3000)
        
        # gt_arts2 = item_trn[item_trn.user_id==test_user].item_id.values
        gt_arts = gt_item.loc[test_user]
        gt_arts = [item2idx[x] for x in gt_arts]
        gt_arts2 = gt_item2.loc[test_user].item_id
        gt_arts2 = [item2idx[x] for x in [gt_arts2]]
        count1+=len(set(gt_arts).intersection(set(idx[0][:])))
        count2+=len(set(gt_arts2).intersection(set(idx[0][:])))
        check+=len(set(gt_arts).intersection(set(gt_arts2)))
        total1+=len(gt_arts)
        total2+=len(gt_arts2)
        pbar.update(1)
        pbar.set_postfix(hit1=count1,hit2=count2,t=total1,tot2=total2)

  2%|▏         | 3613/182741 [00:49<41:10, 72.50it/s, hit1=6482, hit2=1065, t=22817, tot2=3613]


KeyboardInterrupt: 

In [154]:
# test_user_ids = list(item_val.user_id.unique())
# gt_item = item_val.groupby("user_id")["item_id"].apply(list)
# gt_item2 = item_last[item_last.user_id.isin(test_user_ids)].set_index("user_id")
# np.random.shuffle(test_user_ids)
count1 =0 
count2 = 0
total1 = 0
total2 = 0
check = 0
with tqdm(total=len(test_user_ids)) as pbar:
    for test_user in test_user_ids:
        user_embed = model.user_embed(torch.tensor(test_user).cuda()).detach().cpu().numpy()
        # sims2 = np.dot(user_embed,np.array(trn_item_embeds).T)
        # arts_fetch2 = np.random.choice(trn_item_ids,30000)
        # arts_fetch1 = np.array(top_items)[np.argsort(sims2)[-1500:]]
        # arts_fetch2 = np.array(top_items)[np.argsort(sims2)[:148500]]
        sim,idx =  index.search(np.array([user_embed]),3000)
        
        # gt_arts2 = item_trn[item_trn.user_id==test_user].item_id.values
        gt_arts = gt_item.loc[test_user]
        gt_arts = [item2idx[x] for x in gt_arts]
        gt_arts2 = gt_item2.loc[test_user].item_id
        gt_arts2 = [item2idx[x] for x in [gt_arts2]]
        count1+=len(set(gt_arts).intersection(set(idx[0][:])))
        count2+=len(set(gt_arts2).intersection(set(idx[0][:])))
        check+=len(set(gt_arts).intersection(set(gt_arts2)))
        total1+=len(gt_arts)
        total2+=len(gt_arts2)
        pbar.update(1)
        pbar.set_postfix(hit1=count1,hit2=count2,t=total1,tot2=total2)

  1%|          | 1670/182741 [00:22<40:36, 74.32it/s, hit1=2872, hit2=500, t=10101, tot2=1670]


KeyboardInterrupt: 

In [1]:
# test_user_ids = list(item_val.user_id.unique())
# gt_item = item_val.groupby("user_id")["item_id"].apply(list)
# gt_item2 = item_last[item_last.user_id.isin(test_user_ids)].set_index("user_id")
# np.random.shuffle(test_user_ids)
count1 =0 
count2 = 0
total1 = 0
total2 = 0
check = 0
with tqdm(total=len(test_user_ids)) as pbar:
    for test_user in test_user_ids:
        user_embed = model.user_embed(torch.tensor(test_user).cuda()).detach().cpu().numpy()
        # sims2 = np.dot(user_embed,np.array(trn_item_embeds).T)
        # arts_fetch2 = np.random.choice(trn_item_ids,30000)
        # arts_fetch1 = np.array(top_items)[np.argsort(sims2)[-1500:]]
        # arts_fetch2 = np.array(top_items)[np.argsort(sims2)[:148500]]
        sim,idx =  index.search(np.array([user_embed]),3000)
        
        # gt_arts2 = item_trn[item_trn.user_id==test_user].item_id.values
        gt_arts = gt_item.loc[test_user]
        gt_arts = [item2idx[x] for x in gt_arts]
        gt_arts2 = gt_item2.loc[test_user].item_id
        gt_arts2 = [item2idx[x] for x in [gt_arts2]]
        count1+=len(set(gt_arts).intersection(set(idx[0][:])))
        count2+=len(set(gt_arts2).intersection(set(idx[0][:])))
        check+=len(set(gt_arts).intersection(set(gt_arts2)))
        total1+=len(gt_arts)
        total2+=len(gt_arts2)
        pbar.update(1)
        pbar.set_postfix(hit1=count1,hit2=count2,t=total1,tot2=total2)

NameError: name 'tqdm' is not defined

In [113]:
max(idx[0]),len(np.unique(idx[0])),max( [item2idx[x] for x in item_val.item_id])

(328822, 300000, 328822)

In [114]:
len(set(idx[0]).intersection([item2idx[x] for x in item_val.item_id]))

127262

In [116]:
item_trn.item_id.nunique(), item_val.item_id.nunique()

(283142, 146418)

In [85]:
gt_arts,idx[0]

([2013],
 array([   357,    356,    949, ..., 314165, 231708, 142856], dtype=int64))

In [97]:
len(sim[0])

300000

In [27]:
test_user_ids_trn = list(item_trn.user_id.unique())
gt_item_trn = item_trn.groupby("user_id")["item_id"].apply(list)
gt_item_trn2 = item_last[item_last.user_id.isin(test_user_ids_trn)].set_index("user_id")
np.random.shuffle(test_user_ids_trn)
count1 =0 
count2 = 0
total1 = 0
total2 = 0
check = 0
with tqdm(total=len(test_user_ids_trn)) as pbar:
    for test_user in test_user_ids_trn:
        user_embed = model.user_embed(torch.tensor(test_user).cuda()).detach().cpu().numpy()
        # sims2 = np.dot(user_embed,np.array(trn_item_embeds).T)
        # arts_fetch2 = np.random.choice(trn_item_ids,30000)
        # arts_fetch1 = np.array(top_items)[np.argsort(sims2)[-1500:]]
        # arts_fetch2 = np.array(top_items)[np.argsort(sims2)[:148500]]
        sim,idx =  index.search(np.array([user_embed]),3000)
        
        # gt_arts2 = item_trn[item_trn.user_id==test_user].item_id.values
        gt_arts = gt_item_trn.loc[test_user]
        gt_arts = [item2idx[x] for x in gt_arts]
        gt_arts2 = gt_item_trn2.loc[test_user].item_id
        gt_arts2 = [item2idx[x] for x in [gt_arts2]]
        count1+=len(set(gt_arts).intersection(set(idx[0][:3000])))
        count2+=len(set(gt_arts2).intersection(set(idx[0][:3000])))
        check+=len(set(gt_arts).intersection(set(gt_arts2)))
        total1+=len(gt_arts)
        total2+=len(gt_arts2)
        pbar.update(1)
        pbar.set_postfix(hit1=count1,hit2=count2,t=total1,tot2=total2)

  0%|          | 1983/730966 [00:27<2:47:14, 72.65it/s, hit1=8497, hit2=1088, t=13081, tot2=1982]


KeyboardInterrupt: 

In [155]:
# test_user_ids_trn = list(item_trn.user_id.unique())
# gt_item_trn = item_trn.groupby("user_id")["item_id"].apply(list)
# gt_item_trn2 = item_last[item_last.user_id.isin(test_user_ids_trn)].set_index("user_id")
# np.random.shuffle(test_user_ids_trn)
count1 =0 
count2 = 0
total1 = 0
total2 = 0
check = 0
with tqdm(total=len(test_user_ids_trn)) as pbar:
    for test_user in test_user_ids_trn:
        user_embed = model.user_embed(torch.tensor(test_user).cuda()).detach().cpu().numpy()
        # sims2 = np.dot(user_embed,np.array(trn_item_embeds).T)
        # arts_fetch2 = np.random.choice(trn_item_ids,30000)
        # arts_fetch1 = np.array(top_items)[np.argsort(sims2)[-1500:]]
        # arts_fetch2 = np.array(top_items)[np.argsort(sims2)[:148500]]
        sim,idx =  index.search(np.array([user_embed]),3000)
        
        # gt_arts2 = item_trn[item_trn.user_id==test_user].item_id.values
        gt_arts = gt_item_trn.loc[test_user]
        gt_arts = [item2idx[x] for x in gt_arts]
        gt_arts2 = gt_item_trn2.loc[test_user].item_id
        gt_arts2 = [item2idx[x] for x in [gt_arts2]]
        count1+=len(set(gt_arts).intersection(set(idx[0][:3000])))
        count2+=len(set(gt_arts2).intersection(set(idx[0][:3000])))
        check+=len(set(gt_arts).intersection(set(gt_arts2)))
        total1+=len(gt_arts)
        total2+=len(gt_arts2)
        pbar.update(1)
        pbar.set_postfix(hit1=count1,hit2=count2,t=total1,tot2=total2)

  0%|          | 1716/730966 [00:23<2:43:31, 74.32it/s, hit1=6337, hit2=903, t=1e+4, tot2=1716]


KeyboardInterrupt: 

In [158]:
count1 =0 
count2 = 0
total1 = 0
total2 = 0
check = 0
with tqdm(total=len(test_user_ids_trn)) as pbar:
    for test_user in test_user_ids_trn:
        user_embed = model.user_embed(torch.tensor(test_user).cuda()).detach().cpu().numpy()
        # sims2 = np.dot(user_embed,np.array(trn_item_embeds).T)
        # arts_fetch2 = np.random.choice(trn_item_ids,30000)
        # arts_fetch1 = np.array(top_items)[np.argsort(sims2)[-1500:]]
        # arts_fetch2 = np.array(top_items)[np.argsort(sims2)[:148500]]
        sim,idx =  index.search(np.array([user_embed]),3000)
        
        # gt_arts2 = item_trn[item_trn.user_id==test_user].item_id.values
        gt_arts = gt_item_trn.loc[test_user]
        gt_arts = [item2idx[x] for x in gt_arts]
        gt_arts2 = gt_item_trn2.loc[test_user].item_id
        gt_arts2 = [item2idx[x] for x in [gt_arts2]]
        count1+=len(set(gt_arts).intersection(set(idx[0][:3000])))
        count2+=len(set(gt_arts2).intersection(set(idx[0][:3000])))
        check+=len(set(gt_arts).intersection(set(gt_arts2)))
        total1+=len(gt_arts)
        total2+=len(gt_arts2)
        pbar.update(1)
        pbar.set_postfix(hit1=count1,hit2=count2,t=total1,tot2=total2)

  0%|          | 1730/730966 [00:21<2:30:18, 80.86it/s, hit1=5767, hit2=850, t=10175, tot2=1730]


KeyboardInterrupt: 

In [173]:
# test_user_ids_trn = list(item_trn.user_id.unique())
# gt_item_trn = item_trn.groupby("user_id")["item_id"].apply(list)
# gt_item_trn2 = item_last[item_last.user_id.isin(test_user_ids_trn)].set_index("user_id")
# np.random.shuffle(test_user_ids_trn)
count1 =0 
count2 = 0
total1 = 0
total2 = 0
check = 0
with tqdm(total=len(test_user_ids_trn)) as pbar:
    for test_user in test_user_ids_trn:
        user_embed = model.user_embed(torch.tensor(test_user).cuda()).detach().cpu().numpy()
        # sims2 = np.dot(user_embed,np.array(trn_item_embeds).T)
        # arts_fetch2 = np.random.choice(trn_item_ids,30000)
        # arts_fetch1 = np.array(top_items)[np.argsort(sims2)[-1500:]]
        # arts_fetch2 = np.array(top_items)[np.argsort(sims2)[:148500]]
        sim,idx =  index.search(np.array([user_embed]),3000)
        
        # gt_arts2 = item_trn[item_trn.user_id==test_user].item_id.values
        gt_arts = gt_item_trn.loc[test_user]
        gt_arts = [item2idx[x] for x in gt_arts]
        gt_arts2 = gt_item_trn2.loc[test_user].item_id
        gt_arts2 = [item2idx[x] for x in [gt_arts2]]
        count1+=len(set(gt_arts).intersection(set(idx[0][:3000])))
        count2+=len(set(gt_arts2).intersection(set(idx[0][:3000])))
        check+=len(set(gt_arts).intersection(set(gt_arts2)))
        total1+=len(gt_arts)
        total2+=len(gt_arts2)
        pbar.update(1)
        pbar.set_postfix(hit1=count1,hit2=count2,t=total1,tot2=total2)

  0%|          | 1821/730966 [00:22<2:29:06, 81.50it/s, hit1=6065, hit2=897, t=10656, tot2=1821]


KeyboardInterrupt: 

In [21]:
len(trn_item_embeds)

328823

In [273]:
max(top_items),len(set(top_items))

(326699, 150000)

In [287]:
trn_item_embeds[25300]

array([-0.00205872,  0.01067198,  0.00442559,  0.00387845, -0.00243466,
       -0.00018402,  0.00366135, -0.00064728,  0.00664593, -0.00826498,
        0.00831544,  0.00995937,  0.00281455, -0.00441395,  0.00028359,
       -0.00266338, -0.00372765, -0.00231449, -0.00027135, -0.00983913,
        0.00378682, -0.00506463,  0.00596974,  0.00934513, -0.00212543,
       -0.00384673,  0.00536456,  0.00757025, -0.0020385 , -0.00775156,
        0.00721003,  0.00400864,  0.00114823, -0.00374699, -0.0127686 ,
       -0.00113352, -0.00088708,  0.0096148 ,  0.01219288,  0.00373809,
       -0.00394224,  0.00305708, -0.00365596, -0.00378627, -0.00196633,
        0.00321073,  0.01195292, -0.00072244, -0.00443617, -0.00162997,
       -0.00321434, -0.01534081,  0.01623973,  0.00530946,  0.00361922,
        0.00076687, -0.00397851,  0.0058771 ,  0.00251416, -0.00709711,
        0.01413247,  0.00443728, -0.00552901, -0.00657055], dtype=float32)

In [266]:
max(np.array(top_items)[np.argsort(sims2)[-1500:]])

25300

In [260]:
np.array(top_items)[np.argsort(sims2)[-1500:]]+1

array([ 4301, 12498,  3350, ...,    86,   862,   357])

In [ ]:
len(trn_item_embeds)

50000

In [ ]:
len(item_embed_val),len(item_embed_val[0])

(328823, 100)

In [ ]:
item_hist_val_idx = item_hist_val.groupby("user_id")["item_id"].apply(list)

In [ ]:
val_user_id = item_hist_val.user_id.unique()
len(val_user_id)


182741

In [ ]:
user_feature_val = user_feature.loc[val_user_id]
user_feature_val.shape

(182741, 986)

In [ ]:
train_data.item_id.nunique()

50000

In [ ]:
user_embed_val = []


for ii in tqdm(range(0,user_feature_val.shape[0],64)):
    user_feature_t = user_feature_val.iloc[ii:ii+64].values
    # user_feature_t = np.zeros([128,986])
    user_feature_t = user_feature_t.astype(np.float32)
    # print(user_feature_t.shape)
    user_embed = model.user_embed(torch.tensor(user_feature_t).cuda())
    user_embed = user_embed.detach().cpu().numpy()
#     print(user_embed.shape)
    user_embed_val.extend(user_embed)


100%|██████████| 2856/2856 [00:17<00:00, 162.46it/s]


In [ ]:
len(user_embed_val), user_embed_val[182740].shape,user_embed_val[0]

(182741,
 (100,),
 array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan], dtype=float32))

In [ ]:
import faiss
index = faiss.IndexFlatIP(factor_num)
index.add(item_embed_val)

NameError: name 'factor_num' is not defined